# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](https://nlp100.github.io/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

In [2]:
;wget https://nlp100.github.io/data/neko.txt

--2020-11-27 21:36:25--  https://nlp100.github.io/data/neko.txt
nlp100.github.io (nlp100.github.io) をDNSに問いあわせています... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
nlp100.github.io (nlp100.github.io)|185.199.109.153|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 975789 (953K) [text/plain]
`neko.txt' に保存中

     0K .......... .......... .......... .......... ..........  5% 5.08M 0s
    50K .......... .......... .......... .......... .......... 10% 1.01M 0s
   100K .......... .......... .......... .......... .......... 15% 3.08M 0s
   150K .......... .......... .......... .......... .......... 20% 6.07M 0s
   200K .......... .......... .......... .......... .......... 26% 5.28M 0s
   250K .......... .......... .......... .......... .......... 31% 5.13M 0s
   300K .......... .......... .......... .......... .......... 36% 7.59M 0s
   350K .......... .......... .......... .......... .......... 41% 7.39M 0s
   400K .......... .......... .......... .......... ..

In [7]:
;mecab -o ./neko.txt.mecab ./neko.txt

In [13]:
;wc -l neko.txt.mecab

  226266 neko.txt.mecab


In [12]:
;head -10 neko.txt.mecab

一	名詞,数,*,*,*,*,一,イチ,イチ
	記号,一般,*,*,*,*,*
EOS
	記号,一般,*,*,*,*,*
EOS
　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ


***30. 形態素解析結果の読み込み***<br>
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [4]:
filename="neko.txt.mecab"
sentences_ret=[]
open(filename, "r") do f
    sentences=[]
    morphs=[]
    for line in eachline(f)
        if line != "EOS"
            fields=split(line,"\t")
            if length(fields) != 2 || fields[1]==""
                continue
            end
            attr=split(fields[2],",")
            morph=Dict("surface"=>fields[1], "base"=>attr[7],"pos"=>attr[1], "pos1"=>attr[2])
            push!(morphs,morph)
        else
            push!(sentences,morphs)
            morphs=[]
        end
    end
    global sentences_ret=sentences
end
println(length(sentences_ret))
for morph in sentences_ret[3]
    println(morph)
end

9964
Dict("base"=>"　","surface"=>"　","pos"=>"記号","pos1"=>"空白")
Dict("base"=>"吾輩","surface"=>"吾輩","pos"=>"名詞","pos1"=>"代名詞")
Dict("base"=>"は","surface"=>"は","pos"=>"助詞","pos1"=>"係助詞")
Dict("base"=>"猫","surface"=>"猫","pos"=>"名詞","pos1"=>"一般")
Dict("base"=>"だ","surface"=>"で","pos"=>"助動詞","pos1"=>"*")
Dict("base"=>"ある","surface"=>"ある","pos"=>"助動詞","pos1"=>"*")
Dict("base"=>"。","surface"=>"。","pos"=>"記号","pos1"=>"句点")
Dict("base"=>"*","surface"=>"\r","pos"=>"記号","pos1"=>"一般")


In [1]:
# 上記を関数化
function myParse(filename)
    open(filename, "r") do f
        global sentences=[]
        morphs=[]
        for line in eachline(f)
            if line != "EOS"
                fields=split(line,"\t")
                if length(fields) != 2 || fields[1]==""
                    continue
                end
                attr=split(fields[2],",")
                morph=Dict("surface"=>fields[1], "base"=>attr[7],"pos"=>attr[1], "pos1"=>attr[2])
                push!(morphs,morph)
            else
                push!(sentences,morphs)
                morphs=[]
            end
        end
    end
    return sentences
end
filename="neko.txt.mecab"
sentences=myParse(filename)
println(length(sentences))


9964


***31. 動詞***<br>
動詞の表層形をすべて抽出せよ．

In [12]:
global verb=Set()
for sentence in sentences
    for morph in sentence
        if morph["pos"]=="動詞"
            push!(verb,morph["surface"])
        end
    end
end
println(length(verb))

3893


***32. 動詞の原形***<br>
動詞の原形をすべて抽出せよ．


In [14]:
global infinitive=Set()
for sentence in sentences
    for morph in sentence
        if morph["pos"]=="動詞"
            push!(infinitive,morph["base"])
        end
    end
end
println(length(infinitive))

2300


***33. 「Aの B」***<br>
2つの名詞が「の」で連結されている名詞句を抽出せよ.

In [18]:
s=Set()
for sentence in sentences
    for i in 2:length(sentence)
        if sentence[i-1]["pos"]=="名詞" && sentence[i]["surface"]=="の" && sentence[i+1]["pos"]=="名詞"
            push!(s,sentence[i-1]["surface"]*sentence[i]["surface"]*sentence[i+1]["surface"])
        end
    end
end
println(length(s))

4924


***34. 名詞の連接***<br>
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [11]:
global ans_set=Set()
for sentence in sentences
    ans=""
    num=0
    for i in 1:length(sentence)
        for morph in sentence
            if morph["pos"]=="名詞"
                ans*=morph["surface"]
                num+=1
            else
                if num>1
                    push!(ans_set,ans)
                end
                ans=""
                num=0
            end
        end
    end
end
println(length(ans_set))

4457


***35. 単語の出現頻度***<br>
文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．